In [ ]:
CLASSES = 'b'
SIZE = "2k4"

%tensorflow_version 1.x
SIZE = "2k4"
C = "b"
FOLDER = "/" #path to model folder
FOLDS_PATH = "/" #path where the folds are saved
DATA_FOLDER = "/" #path where the entire dataset is saved

CLASS_NAME = {2:['Sem Risco', 'Risco Potencial'], 3:['Sem Risco', 'Risco Potencial', 'Risco Alto']}

In [ ]:
from time import ctime

import json
import os
import numpy as np
import pandas as pd
import sklearn.svm as svm

from scipy import stats
from sklearn import preprocessing
from sklearn.metrics import make_scorer, roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV 
from sklearn.svm import SVC as svc 

In [ ]:
created_at = ctime()
created_at = created_at.replace(' ', '-')

#Definindo funções de métrica

In [ ]:
accuracy = make_scorer(precision_score)
f1 = make_scorer(f1_score, average='macro')
precision = make_scorer(precision_score, average='macro')
recall = make_scorer(recall_score, average='macro')

scorer = make_scorer(f1_score, average='macro')

In [ ]:
def evaluate(clf, X, y):
  y_pred = clf.predict(X)
  acc = accuracy_score(y, y_pred)
  print(acc)
  result = [acc, precision(clf, X, y), recall(clf, X, y), f1(clf, X,y)]
  return result

In [ ]:
params = {}

#Carregando dataset

In [ ]:
df_train = pd.read_csv(DATA_FOLDER+"ideacao-{}-{}-train.csv".format(SIZE, CLASSES), sep='\t')
df_test = pd.read_csv(DATA_FOLDER+"ideacao-{}-{}-test.csv".format(SIZE, CLASSES), sep='\t')


cols = [str(i) for i in range(50)]

df_train['embedding'] = df_train.loc[:,'0':'49'].values.tolist()
df_test['embedding'] = df_test.loc[:,'0':'49'].values.tolist()

df_train.drop(cols, axis=1, inplace=True)
df_test.drop(cols, axis=1, inplace=True)


train_features = df_train['embedding']
train_labels = df_train['y']

test_features = df_test['embedding']
test_labels = df_test['y']


X = []
for t in train_features:
  X.append(np.array(t))
train_features = np.array(X)

X = []
for t in test_features:
  X.append(np.array(t))
test_features = np.array(X)


#Definindo modelo

In [ ]:
def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10, 100, 1000]    
    gammas = [0.001, 0.01, 0.1, 1, 10, 100]    
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svc(kernel='rbf'), param_grid, cv=nfolds, scoring=scorer, verbose=1)
    grid_search.fit(X, y)    
    grid_search.best_params_
    return grid_search

In [ ]:
grid_search = svc_param_selection(train_features, train_labels, 10)

In [ ]:
params['grid'] = grid_search.best_params_

In [ ]:
clf = svc(kernel='rbf', C=params['grid']['C'], gamma=params['grid']['gamma']) 
clf.fit(train_features, train_labels)
grid_res = evaluate(clf, test_features, test_labels)

#Salvando Resultados

In [ ]:
n = "grid"
name = "{}-{}-{}-{}-class-results.csv".format(n, SIZE, created_at, CLASSES)
with open(os.path.join(FOLDER, 'results', name), 'w') as f:
  f.write("acc, prec, rec, f1\n")
  f.write("{},{},{},{}\n".format(grid_res[0], grid_res[1], grid_res[2], grid_res[3]))  

#Random Search

In [ ]:
mdl = svc(probability = True, random_state = 1)
rand_list = {"C": stats.uniform(2, 10),
             "gamma": stats.uniform(0.1, 1)}
              
rand_search = RandomizedSearchCV(mdl, param_distributions = rand_list, n_iter = 20, n_jobs = 4, cv = 3, random_state = 2017, scoring = scorer) 
rand_search.fit(train_features, train_labels) 
rand_search.best_params_

In [ ]:
bp = rand_search.best_params_

In [ ]:
clf = svc(kernel='rbf', C=bp['C'], gamma=bp['gamma']) 
clf.fit(train_features, train_labels)
rand_res = evaluate(clf, test_features, test_labels)

#Salvando Resultados

In [ ]:
n = "rand"

name = "{}-{}-{}-{}-class-results.csv".format(n, SIZE, created_at, CLASSES)
with open(os.path.join(FOLDER, 'results', name), 'w') as f:
  f.write("acc, prec, rec, f1\n")
  f.write("{},{},{},{}\n".format(rand_res[0], rand_res[1], rand_res[2], rand_res[3]))  

In [ ]:
params['rand'] = rand_search.best_params_

In [ ]:
with open(FOLDER+"params-{}.json".format(created_at), "w") as f:
  json.dump(params, f)